In [ ]:
!pip install chart-studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### import lib

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

#tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import time
import string

import chart_studio.plotly
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
#%plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import os

### dataset

In [ ]:
lines =open('ben.txt', encoding='UTF-8').read().strip().split('\n')
lines[2:12]

['Go.\tযা।\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545006 (tanay)',
 'Run!\tপালাও!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5548781 (tanay)',
 'Run!\tপালান!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5548783 (tanay)',
 'Who?\tকে?\tCC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #5548787 (tanay)',
 'Fire!\tআগুন!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #3232240 (tanay)',
 'Help!\tবাঁচাও!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #5548780 (tanay)',
 'Help!\tবাঁচান!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #5548782 (tanay)',
 'Stop!\tথামুন!\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (CM) & #5545000 (tanay)',
 'Stop!\tথামো!\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (CM) & #5545001 (tanay)',
 'Stop!\tথাম!\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (CM) & #5545002 (tanay)']

In [ ]:
print("total number of records: ",len(lines))

total number of records:  4619


In [ ]:
exclude = set(string.punctuation) # Set of all special characters
remove_digits = str.maketrans('', '', string.digits) # Set of all digits

In [ ]:
lines[2:12]

['Go.\tযা।\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545006 (tanay)',
 'Run!\tপালাও!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5548781 (tanay)',
 'Run!\tপালান!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5548783 (tanay)',
 'Who?\tকে?\tCC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #5548787 (tanay)',
 'Fire!\tআগুন!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #3232240 (tanay)',
 'Help!\tবাঁচাও!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #5548780 (tanay)',
 'Help!\tবাঁচান!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #5548782 (tanay)',
 'Stop!\tথামুন!\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (CM) & #5545000 (tanay)',
 'Stop!\tথামো!\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (CM) & #5545001 (tanay)',
 'Stop!\tথাম!\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (CM) & #5545002 (tanay)']

### **Function to preprocess English sentence**

In [ ]:
def preprocess_eng_sentence(sent):
    '''Function to preprocess English sentence'''
    sent = sent.lower() # lower casing
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits) # remove the digits
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

### **Function to preprocess bengali sentence** 

In [ ]:
def preprocess_port_sentence(sent):
    '''Function to preprocess Portuguese sentence'''
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in exclude)
    #sent = re.sub("[२३०८१५७९४६]", "", sent) # remove the digits
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

### **Generate pairs of cleaned English and bengali sentences with start and end tokens added**

In [ ]:
# Generate pairs of cleaned English and Portuguese sentences
sent_ben = []
for line in lines:
    sent_pair = []
    eng = line.rstrip().split('\t')[0]
    port = line.rstrip().split('\t')[1]
    eng = preprocess_eng_sentence(eng)
    sent_pair.append(eng)
    port = preprocess_port_sentence(port)
    sent_pair.append(port)
    sent_ben.append(sent_pair)
sent_ben[2:12]

[['<start> go <end>', '<start> যা। <end>'],
 ['<start> run <end>', '<start> পালাও <end>'],
 ['<start> run <end>', '<start> পালান <end>'],
 ['<start> who <end>', '<start> কে <end>'],
 ['<start> fire <end>', '<start> আগুন <end>'],
 ['<start> help <end>', '<start> বাঁচাও <end>'],
 ['<start> help <end>', '<start> বাঁচান <end>'],
 ['<start> stop <end>', '<start> থামুন <end>'],
 ['<start> stop <end>', '<start> থামো <end>'],
 ['<start> stop <end>', '<start> থাম <end>']]

### **Create a class to map every word to an index and vice-versa for any given vocabulary**

In [ ]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)

### tokenization and padding

In [ ]:
def load_dataset(ben, num_examples):
    # ben => already created cleaned input, output ben

    # index language using the class defined above    
    inp_lang = LanguageIndex(en for en, ma in ben)
    targ_lang = LanguageIndex(ma for en, ma in ben)
    
    # Vectorize the input and target languages
    
    # English sentences
    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ma in ben]
    
    # Marathi sentences
    target_tensor = [[targ_lang.word2idx[s] for s in ma.split(' ')] for en, ma in ben]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [ ]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_ben, len(lines))

### **Creating training and validation sets using an 80-20 split**

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1, random_state = 101)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(4157, 4157, 462, 462)

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

*We'll be using GRUs instead of LSTMs as we only have to create one state and implementation would be easier.*

### **Create GRU units**

In [ ]:
def gru(units):

    return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

### **The next step is to define the encoder and decoder network**

The input to the encoder will be the sentence in English and the output will be the hidden state and cell state of the GRU

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

The next step is to define the decoder. The decoder will have two inputs: the hidden state and cell state from the encoder and the input sentence, which actually will be the output sentence with a token appended at the beginning.



In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
            
    def call(self, x, hidden, enc_output):

        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
     
             
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

*Create encoder and decoder objects from their respective classes.*

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

### **Define the optimizer and the loss function.**

In [ ]:
optimizer = tf.optimizers.Adam()

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

### **Training the Model**

In [ ]:
EPOCHS = 24

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.0838
Epoch 1 Loss 1.6495
Time taken for 1 epoch 37.529327392578125 sec

Epoch 2 Batch 0 Loss 1.3780
Epoch 2 Loss 1.3888
Time taken for 1 epoch 22.524174690246582 sec

Epoch 3 Batch 0 Loss 1.2680
Epoch 3 Loss 1.2612
Time taken for 1 epoch 23.364022970199585 sec

Epoch 4 Batch 0 Loss 1.1471
Epoch 4 Loss 1.1619
Time taken for 1 epoch 24.061120986938477 sec

Epoch 5 Batch 0 Loss 1.1046
Epoch 5 Loss 1.0532
Time taken for 1 epoch 22.71214199066162 sec

Epoch 6 Batch 0 Loss 0.9438
Epoch 6 Loss 0.9411
Time taken for 1 epoch 22.97822618484497 sec

Epoch 7 Batch 0 Loss 0.7888
Epoch 7 Loss 0.8262
Time taken for 1 epoch 22.6613986492157 sec

Epoch 8 Batch 0 Loss 0.7028
Epoch 8 Loss 0.7089
Time taken for 1 epoch 23.372658729553223 sec

Epoch 9 Batch 0 Loss 0.6162
Epoch 9 Loss 0.6045
Time taken for 1 epoch 22.659663200378418 sec

Epoch 10 Batch 0 Loss 0.5055
Epoch 10 Loss 0.5029
Time taken for 1 epoch 22.737654209136963 sec

Epoch 11 Batch 0 Loss 0.3885
Epoch 11 Loss 0.4156
Ti

### **Restoring the latest checkpoint**

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### **Inference setup and testing**

In [ ]:
def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '
    sentence = sentence[:-1]
    
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

### **Function to predict (translate) a randomly selected test point**

In [ ]:
def predict_random_val_sentence():
    actual_sent = ''
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    result, sentence, attention_plot = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
    print('Input: {}'.format(sentence[8:-6]))
    print('Predicted translation: {}'.format(result[:-6]))
    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]
    print('Actual translation: {}'.format(actual_sent))
    attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]

    # use plotly to generate the heat map
    trace = go.Heatmap(z = attention_plot, x = sentence, y = result, colorscale='greens')
    data=[trace]
    iplot(data)

In [ ]:
predict_random_val_sentence()

Input: i agree with you
Predicted translation: আমি আপনার সাথে একমত। 
Actual translation: আমি তোমার সাথে একমত।


In [ ]:
predict_random_val_sentence()

Input: finish it before you go to bed
Predicted translation: ঘুমাতে যাওয়ার আগে ওটা শেষ করে ফেল্। 
Actual translation: ঘুমাতে যাওয়ার আগে ওটা শেষ করে ফেলুন।


In [ ]:
predict_random_val_sentence()

Input: i tried
Predicted translation: আমি চেষ্টা করেছি। 
Actual translation: আমি চেষ্টা করেছিলাম।
